# Covid-19 Data Analysis&Data Modeling

### Importing Libraries

In [257]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

### Importing time series dataset
Dataset is downloaded from JHU git repository

In [220]:
confirmed=pd.read_csv(r"https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true")
deaths= pd.read_csv(r"https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv?raw=true")
recover=pd.read_csv(r"https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv?raw=true")

In [221]:
print(confirmed.shape)
print(deaths.shape)
print(recover.shape)

(275, 461)
(275, 461)
(260, 461)


In [222]:
confirmed.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '4/13/21', '4/14/21', '4/15/21', '4/16/21', '4/17/21', '4/18/21',
       '4/19/21', '4/20/21', '4/21/21', '4/22/21'],
      dtype='object', length=461)

### Relabeling the dataset

In [223]:
con_columns=list(confirmed.columns)

con_columns.remove('Province/State')
con_columns.remove('Lat')
con_columns.remove('Long')
con_columns.remove("Country/Region")

con_days=[]
for i in range(1,len(con_columns)+1):
    con_days.append(i)


con_days.insert(0,'Province/State')
con_days.insert(1,"Country/Region")
con_days.insert(2,'Lat')
con_days.insert(3,'Long')

confirmed.columns=con_days      

In [224]:
deaths_columns=list(deaths.columns)

deaths_columns.remove('Province/State')
deaths_columns.remove('Lat')
deaths_columns.remove('Long')
deaths_columns.remove("Country/Region")

deaths_days=[]
for i in range(1,len(deaths_columns)+1):
    deaths_days.append(i)


deaths_days.insert(0,'Province/State')
deaths_days.insert(1,"Country/Region")
deaths_days.insert(2,'Lat')
deaths_days.insert(3,'Long')

deaths.columns=deaths_days      

In [225]:
recovers_columns=list(recovers.columns)

recovers_columns.remove('Province/State')
recovers_columns.remove('Lat')
recovers_columns.remove('Long')
recovers_columns.remove("Country/Region")

recovers_days=[]
for i in range(1,len(recovers_columns)+1):
    recovers_days.append(i)


recovers_days.insert(0,'Province/State')
recovers_days.insert(1,"Country/Region")
recovers_days.insert(2,'Lat')
recovers_days.insert(3,'Long')

recover.columns=recovers_days   


In [226]:
confirmed.head()

,Province/State,Country/Region,Lat,Long,1,2,3,4,5,6,...,448,449,450,451,452,453,454,455,456,457
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,57364,57492,57534,57612,57721,57793,57898,58037,58214,58312
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,128752,128959,129128,129307,129456,129594,129694,129842,129980,130114
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,118799,118975,119142,119323,119486,119642,119805,119992,120174,120363
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,12614,12641,12641,12712,12771,12805,12805,12874,12917,12942
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,23697,23841,23951,24122,24300,24389,24518,24661,24883,25051


In [227]:
confirmed.describe()

,Lat,Long,1,2,3,4,5,6,7,8,...,448,449,450,451,452,453,454,455,456,457
count,273.000000,273.000000,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000,...,2.750000e+02,2.750000e+02,2.750000e+02,2.750000e+02,2.750000e+02,2.750000e+02,2.750000e+02,2.750000e+02,2.750000e+02,2.750000e+02
mean,20.534804,23.028143,2.025455,2.381818,3.421818,5.210909,7.701818,10.643636,20.283636,22.425455,...,4.997921e+05,5.027631e+05,5.057331e+05,5.088417e+05,5.117073e+05,5.142000e+05,5.167215e+05,5.198285e+05,5.230648e+05,5.263369e+05
std,25.194592,73.596166,26.830287,26.928190,33.524534,46.659184,65.206845,87.856574,215.590294,216.912048,...,2.314655e+06,2.325437e+06,2.336484e+06,2.348518e+06,2.359359e+06,2.369276e+06,2.379901e+06,2.392283e+06,2.405482e+06,2.418723e+06
min,-51.796300,-178.116500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00
25%,5.152149,-19.020800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.068000e+03,1.085000e+03,1.103000e+03,1.131500e+03,1.137500e+03,1.138000e+03,1.139500e+03,1.140500e+03,1.143000e+03,1.145500e+03
50%,21.694000,20.939400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.754900e+04,1.770600e+04,1.780600e+04,1.788300e+04,1.796500e+04,1.808100e+04,1.808100e+04,1.808100e+04,1.817500e+04,1.819200e+04
75%,41.112900,84.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.087260e+05,2.100440e+05,2.112460e+05,2.126530e+05,2.138955e+05,2.145340e+05,2.151175e+05,2.163705e+05,2.176155e+05,2.189595e+05
max,71.706900,178.065000,444.000000,444.000000,549.000000,761.000000,1058.000000,1423.000000,3554.000000,3554.000000,...,3.134598e+07,3.142136e+07,3.149565e+07,3.157564e+07,3.162801e+07,3.167003e+07,3.173796e+07,3.179924e+07,3.186209e+07,3.192935e+07


In [228]:
confirmed.isnull()

,Province/State,Country/Region,Lat,Long,1,2,3,4,5,6,...,448,449,450,451,452,453,454,455,456,457
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
271,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
272,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
273,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Retriving the multiple rows of perticular country of confirmed cases

In [229]:
confirmed[confirmed['Country/Region']=="Australia"]

,Province/State,Country/Region,Lat,Long,1,2,3,4,5,6,...,448,449,450,451,452,453,454,455,456,457
8,Australian Capital Territory,Australia,-35.4735,149.0124,0,0,0,0,0,0,...,123,123,123,123,123,123,123,123,123,123
9,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,3,4,...,5347,5356,5363,5370,5376,5384,5387,5395,5402,5418
10,Northern Territory,Australia,-12.4634,130.8456,0,0,0,0,0,0,...,112,113,113,113,119,119,129,132,136,149
11,Queensland,Australia,-27.4698,153.0251,0,0,0,0,0,0,...,1508,1509,1515,1516,1518,1518,1518,1519,1520,1524
12,South Australia,Australia,-34.9285,138.6007,0,0,0,0,0,0,...,675,677,678,682,682,682,688,691,693,702
13,Tasmania,Australia,-42.8821,147.3272,0,0,0,0,0,0,...,234,234,234,234,234,234,234,234,234,234
14,Victoria,Australia,-37.8136,144.9631,0,0,0,0,1,1,...,20487,20492,20492,20494,20494,20498,20499,20502,20504,20506
15,Western Australia,Australia,-31.9505,115.8605,0,0,0,0,0,0,...,964,965,966,967,973,975,978,980,982,982


In [230]:
confirmed['Country/Region'].nunique()

192

### Removing Null values with Blank

In [231]:
confirmed=confirmed.replace(np.nan,'',regex=True)

deaths=deaths.replace(np.nan,'',regex=True)

recovers=recover.replace(np.nan,'',regex=True)

In [232]:
confirmed.isnull() #Same will be displayed for deaths and recovered cases

,Province/State,Country/Region,Lat,Long,1,2,3,4,5,6,...,448,449,450,451,452,453,454,455,456,457
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
271,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
272,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
273,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Total confirmed cases globally

In [233]:
confirmed_ts=confirmed.drop(['Lat','Long','Country/Region','Province/State'],axis=1)

confirmed_ts_summery=confirmed_tc.sum()

confirmed_ts_summery


1            557
2            655
3            941
4           1433
5           2118
         ...    
453    141404998
454    142098420
455    142952845
456    143842820
457    144742651
Length: 457, dtype: int64

In [234]:
confirmed_ts_summery.index

Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       448, 449, 450, 451, 452, 453, 454, 455, 456, 457],
      dtype='object', length=457)

### Visualizing the total confirmed cases

In [235]:
fig_1=go.Figure(data=go.Scatter(x=confirmed_ts_summery.index,y=confirmed_ts_summery.values,mode='lines+markers'))

fig_1.update_layout(title='Total Confirmed Cases Globally', yaxis_title='Confirmed Cases', xaxis_tickangle=315)
fig_1.show()

## Covid-19 Case Status

In [236]:
#Here ts used for time series and agg for aggregated

confirmed_agg_ts=confirmed.drop(['Lat','Long','Country/Region','Province/State'],axis=1).sum()
deaths_agg_ts=deaths.drop(['Lat','Long','Country/Region','Province/State'],axis=1).sum()
recovered_agg_ts=recover.drop(['Lat','Long','Country/Region','Province/State'],axis=1).sum()


In [237]:
confirmed_agg_ts.index

Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       448, 449, 450, 451, 452, 453, 454, 455, 456, 457],
      dtype='object', length=457)

In [238]:
deaths_agg_ts.index

Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       448, 449, 450, 451, 452, 453, 454, 455, 456, 457],
      dtype='object', length=457)

In [239]:
recovered_agg_ts.index

Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       448, 449, 450, 451, 452, 453, 454, 455, 456, 457],
      dtype='object', length=457)

### Deriving Active case time serier data from confirmed cases and deaths
Active cases are nothing diffrence of confirmed cases, deaths cases and recover cases

In [240]:
active_case_agg_ts=pd.Series(
    [x1 - x2-x3 for (x1,x2,x3)in zip(confirmed_agg_ts.values,deaths_agg_ts.values,recovers_agg_ts.values,)], index=confirmed_agg_ts.index)

In [241]:
active_case_agg_ts.describe()

count    4.570000e+02
mean     1.742099e+07
std      1.874005e+07
min      5.100000e+02
25%      2.605562e+06
50%      8.121226e+06
75%      3.390763e+07
max      5.854263e+07
dtype: float64

In [242]:
active_case_agg_ts.isnull()

1      False
2      False
3      False
4      False
5      False
       ...  
453    False
454    False
455    False
456    False
457    False
Length: 457, dtype: bool

## Visualizing the data

In [245]:
con_fig=go.Figure(data=go.Scatter(x=confirmed_agg_ts.index,y=confirmed_agg_ts.values,mode='lines+markers',marker=dict(color='blue')))
con_fig.update_layout(title='Confirmed Cases Globally',yaxis_title='Confirmed Cases',xaxis_tickangle=-270)

con_fig.show()

In [246]:
recover_fig=go.Figure(data=go.Scatter(x=recovered_agg_ts.index,y=recovered_agg_ts.values,mode='lines+markers',marker=dict(color='green')))
recover_fig.update_layout(title='Recovered Cases Globally',yaxis_title='Recovered Cases',xaxis_tickangle=-270)

recover_fig.show()

In [243]:
active_fig=go.Figure(data=go.Scatter(x=active_case_agg_ts.index,y=active_case_agg_ts.values,mode='lines+markers',marker=dict(color='orange')))
active_fig.update_layout(title='Active Cases Globally',yaxis_title='Active Cases',xaxis_tickangle=-270)

active_fig.show()

In [312]:
deaths_fig=go.Figure(data=go.Scatter(x=deaths_agg_ts.index,y=deaths_agg_ts.values,mode='lines+markers',marker=dict(color='red')))
deaths_fig.update_layout(title='Death Cases Globally',yaxis_title='Death Cases',xaxis_tickangle=-270)

deaths_fig.show()

In [311]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=confirmed_agg_ts.index,y=active_case_agg_ts.values,name="Active Cases",marker=dict(color='orange')))
fig.add_trace(go.Scatter(x=confirmed_agg_ts.index,y=confirmed_agg_ts.values,name="Confirmed Cases",marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=confirmed_agg_ts.index,y=deaths_agg_ts.values,name="Deaths",marker=dict(color='red')))
fig.add_trace(go.Scatter(x=confirmed_agg_ts.index,y=recovers_agg_ts.values,name="Recover Cases",marker=dict(color='green')))

fig.show()

# Deriving the fatality rate of Covid-19
Crafting data to plot a pie chart

confirmed_agg_ts will be used to get total number of cases and just like confirmed_agg_ts, recovered_agg_ts, deaths_agg_ts and active_case_agg_ts summery will be used

sum of all the values in each data set will be used to plot pie chart

In [279]:
confirmed_case=confirmed_agg_ts.values.sum()
deaths_case=deaths_agg_ts.values.sum()
recovered_case=recovered_agg_ts.values.sum()
active_case=active_case_agg_ts.values.sum()

data_lst=[]
data_lst.append(confirmed_case)
data_lst.append(deaths_case)
data_lst.append(recovered_case)
data_lst.append(active_case)
#df_pie refers to dataframe for piechart
df_pie=pd.Series(data_lst)


In [280]:
df_pie.head()

0    20369645948
1      501397535
2    11906855886
3     7961392527
dtype: int64

In [281]:
#Renaming column of new dataframe
lable_pie=['Confirmed_cases','Death_cases','recovered_cases','active_cases']
df_pie.columns=lable_pie


In [282]:
print(df_pie.columns,df_pie.values)

['Confirmed_cases', 'Death_cases', 'recovered_cases', 'active_cases'] [20369645948   501397535 11906855886  7961392527]


In [327]:
pie_chart=go.Pie(labels=df_pie.columns,values=df_pie.values)
draw_pie_chart=go.Figure(data=[pie_chart])

draw_pie_chart.update_layout(title="Covid-19 Status in fatality globally")
draw_pie_chart.update_traces(marker=dict(colors=['blue','red','green','orange']))

draw_pie_chart.show()

From the above pie chart it is clearly visible that death rate of the covid-19 is 1.23%